# 📊 DataLens AI - Intelligent Data Analytics Agent

An autonomous AI-powered data analytics system that transforms raw datasets (CSV/Excel) into professional visualizations and interactive dashboards. Built with Google's Gemini API, this agent intelligently analyzes data, performs automated cleaning, and generates comprehensive visual insights.

## 🚀 Workflow Pipeline
**Setup Environment** → **Initialize Gemini AI** → **Load Data** → **AI-Powered Analysis & Cleaning** → **Generate Visualizations & Dashboard**

## 🎯 Core Capabilities
- **AI-Driven Intelligence**: Leverages Gemini API for automated data quality assessment and insights generation
- **Interactive Data Upload**: Seamless file upload widget supporting CSV and Excel formats
- **Automated Cleaning**: Generates and applies intelligent cleaning code based on data profiling
- **Smart Outlier Handling**: Uses statistical capping methods to preserve data integrity
- **Production-Ready Output**: Delivers ML-ready datasets with proper encoding and standardization
- **Professional Visualizations**: Creates publication-quality charts and interactive dashboards

# Cell 1 : Environment Setup
Installing all required dependencies for the data analytics agent including data processing libraries (pandas, numpy), visualization tools (matplotlib, seaborn, plotly), machine learning utilities (scikit-learn), and Google Generative AI SDK for agent capabilities.

In [ ]:
!pip install --upgrade pip --quiet  # Update pip to latest version
!pip install pandas numpy matplotlib seaborn plotly scikit-learn ipywidgets jsonschema \
            google-generativeai google-auth google-auth-oauthlib --quiet  # Data processing, visualization, and Gemini API libraries
!pip install openpyxl xlrd jupyterlab --quiet  # Excel file handling libraries

# Cell 2: Import Libraries and Verify Installation
Importing all essential libraries and verifying successful installation. This cell performs safe imports with error handling, checks for missing packages, displays library versions, and configures visualization settings for the notebook environment.

In [ ]:
# Import cell

import importlib
import sys
import warnings

# List of required packages for verification
expected = [
    "pandas", "numpy", "matplotlib", "seaborn", "plotly",
    "google.genai", "ipywidgets", "jsonschema", "sklearn", "tqdm"
]

missing = []
for pkg in expected:
    try:
        importlib.import_module(pkg)  # Check if package is importable
    except Exception as exc:
        missing.append((pkg, str(exc)))

if missing:
    print("⚠️ Some packages are not importable. Please run the install cell, restart the runtime, and run this cell again.\n")
    for pkg, err in missing:
        print(f"- {pkg}: {err}")
    # Stop here to avoid confusing errors
else:
    # Import all libraries with proper aliases
    try:
        import pandas as pd
        import numpy as np
        import matplotlib
        import matplotlib.pyplot as plt

        # Enable inline plotting for IPython/Colab environments
        try:
            ip = get_ipython()
            ip.run_line_magic("matplotlib", "inline")
        except NameError:
            pass  # Not in IPython environment

        import seaborn as sns
        import plotly
        import plotly.express as px
        import plotly.io as pio

        # Set Plotly renderer for Colab compatibility
        try:
            if "colab" in pio.renderers:
                pio.renderers.default = "colab"
            else:
                pio.renderers.default = pio.renderers.default
        except Exception:
            pass

        import google.genai as genai
        import ipywidgets as widgets
        import jsonschema as _jsonschema
        from jsonschema import validate, Draft7Validator
        import sklearn
        from tqdm import tqdm

    except Exception as e:
        print("⚠️ Import error after initial checks:", e)
        print("Tip: re-run the install cell, restart the runtime, then run this cell again.")
        raise

    # Display version information for debugging
    def safe_ver(m):
        return getattr(m, "__version__", getattr(m, "version", "unknown"))

    print("\nLibrary versions:")
    print("pandas        :", safe_ver(pd))
    print("numpy         :", safe_ver(np))
    print("matplotlib    :", safe_ver(matplotlib))
    print("seaborn       :", safe_ver(sns))
    print("plotly        :", safe_ver(plotly))
    print("google.genai  :", getattr(genai, "__version__", "unknown"))
    print("ipywidgets    :", safe_ver(widgets))
    print("jsonschema    :", safe_ver(_jsonschema))
    print("scikit-learn  :", safe_ver(sklearn))
    print("tqdm          :", safe_ver(tqdm))

    # Configure default visualization settings
    sns.set(style="whitegrid")
    warnings.filterwarnings("ignore")  # Suppress non-critical warnings
    print("\n✅ Imports complete — ready for the next step.")

# Cell 3: API Key Configuration
Securely retrieving the Gemini API key from Google Colab's userdata storage. This ensures credentials are not hardcoded in the notebook for security best practices.

In [ ]:
from google.colab import userdata
api_key = userdata.get("GEMINI_API_KEY")  # Fetch API key from Colab secrets

# Cell 4: Initialize Gemini AI Client
Setting up the Gemini AI client using the secure API authentication. This creates a reusable client instance for all subsequent AI-powered data analysis operations.

In [ ]:
# Step 2 — Correct Gemini API Setup (google-genai v1.49+)

import google.genai as genai
from google.colab import userdata

def init_gemini():
    """
    Initialize Gemini client with API key from Colab secrets.
    Returns configured client instance for AI operations.
    """
    api_key = userdata.get("GEMINI_API_KEY")  # Retrieve API key securely
    if not api_key:
        raise ValueError("❌ Gemini API key not found in Colab Secrets!")

    client = genai.Client(api_key=api_key)  # Create Gemini client instance
    print("✅ Gemini Client created (secure).")

    return client

# Initialize client for global use
client = init_gemini()

# Cell 5: Data Upload and Automated Summary Generation
Interactive file upload interface that accepts CSV/Excel files and automatically generates a comprehensive dataset summary. This summary includes statistical metrics, missing value analysis, and data type information for AI-powered processing.

In [ ]:
# Step 3 — Upload Dataset + Auto Summary

import pandas as pd
import numpy as np
from google.colab import files

def upload_dataset():
    """
    Interactive file upload widget for CSV/Excel files.
    Returns loaded DataFrame with preview and basic info.
    """
    print("📁 Upload your CSV or Excel file:")
    uploaded = files.upload()

    file_name = next(iter(uploaded))

    # Load dataset based on file extension
    if file_name.lower().endswith(".csv"):
        df = pd.read_csv(file_name)
    elif file_name.lower().endswith((".xlsx", ".xls")):
        df = pd.read_excel(file_name)
    else:
        raise ValueError("❌ Unsupported file format. Upload CSV or Excel.")

    print("\n✅ File Loaded Successfully!\n")

    # Display preview and basic statistics
    display(df.head())
    print("\n📏 Shape:", df.shape)
    print("\n📊 Data Types:\n", df.dtypes)

    return df

# -------- Summary Generator -------- #

def generate_dataset_summary(df):
    """
    Generate comprehensive column-wise summary for AI analysis.
    Includes stats, missing values, and data type information.
    """
    summary = {}

    for col in df.columns:
        col_data = df[col]
        info = {
            "dtype": str(col_data.dtype),
            "missing_count": int(col_data.isna().sum()),
            "missing_percent": float((col_data.isna().mean() * 100)),
            "unique_count": int(col_data.nunique())
        }

        # Calculate numeric statistics
        if pd.api.types.is_numeric_dtype(col_data):
            info["min"] = float(col_data.min())
            info["max"] = float(col_data.max())
            info["mean"] = float(col_data.mean())
            info["std"] = float(col_data.std())

        # Extract categorical samples
        if pd.api.types.is_object_dtype(col_data) or pd.api.types.is_categorical_dtype(col_data):
            info["unique_values"] = col_data.dropna().unique().tolist()[:10]  # Top 10 samples

        summary[col] = info

    return summary


# -------- Run Upload + Summary -------- #

df = upload_dataset()  # Load dataset interactively
dataset_summary = generate_dataset_summary(df)  # Generate AI-ready summary

print("\n📘 Dataset Summary for LLM:")
dataset_summary

# Cell 6: AI-Powered Data Cleaning Agent
Implements an intelligent cleaning agent that analyzes dataset summary and generates customized cleaning recommendations with executable Python code. Uses Gemini AI to identify data quality issues and propose automated solutions.

In [ ]:
# Step 4 — Simple Cleaning Agent (NO JSON)

def build_cleaning_prompt(summary):
    """
    Construct prompt for Gemini to analyze data quality issues.
    Returns structured prompt requesting problems, recommendations, and code.
    """
    return f"""
You are a professional data cleaning agent.

Analyze the dataset summary below and produce:

1. A list of problems found (missing values, negative values, inconsistent categories, invalid ranges, dtype issues, outliers).
2. Recommended cleaning steps.
3. Python code that cleans the dataframe (df) and outputs df_clean.

Notes:
- Do NOT use JSON.
- Do NOT use markdown.
- Only plain text and Python code.
- Python code must assume the dataframe is already named df.
- Cleaning code must output df_clean.

Dataset Summary:
{summary}
"""


def ask_gemini_cleaning(prompt):
    """
    Send cleaning analysis prompt to Gemini API.
    Returns AI-generated cleaning recommendations and code.
    """
    try:
        response = client.models.generate_content(
            model="gemini-2.5-pro",  # Use latest Gemini model
            contents=prompt,
            config={"response_mime_type": "text/plain"}  # Plain text response
        )
        return response.text
    except Exception as e:
        return f"Error: {e}"

# Cell 7: Execute AI Data Cleaning Analysis
Generates and sends the cleaning prompt to Gemini AI, receiving intelligent analysis of data quality issues along with automated cleaning recommendations and executable code.

In [ ]:
cleaning_prompt = build_cleaning_prompt(dataset_summary)  # Create cleaning prompt with dataset summary
cleaning_output = ask_gemini_cleaning(cleaning_prompt)  # Get AI-generated cleaning recommendations

print(cleaning_output)  # Display problems, recommendations, and cleaning code

# Cell 8: Intelligent Outlier Capping
Applies statistical outlier capping to numerical columns using the 99th percentile method. This preserves data points while reducing extreme values, ensuring dataset integrity for analysis and modeling.

In [ ]:
import pandas as pd
import numpy as np

# Create a copy of the dataframe to work on
df_clean = df.copy()

# Dynamically identify numerical columns for outlier capping
# Exclude identifier columns (like 'Store') and binary flags (like 'Holiday_Flag')
numeric_cols_for_capping = [
    col for col in df_clean.select_dtypes(include=np.number).columns
    if col not in ['Store', 'Holiday_Flag']  # Adjust exclusions based on dataset
]

print(f"Applying 99th percentile capping to columns: {numeric_cols_for_capping}")

# Apply 99th percentile capping to reduce outlier impact
for col in numeric_cols_for_capping:
    upper_cap = df_clean[col].quantile(0.99)  # Calculate 99th percentile threshold
    df_clean[col] = df_clean[col].clip(upper=upper_cap)  # Cap values at threshold
    print(f"  - Capped '{col}' at: {upper_cap:.2f}")

print("\n✅ Outlier capping applied to selected numerical columns.")

# Cell 9: Comprehensive Data Preprocessing Pipeline
Complete data preprocessing including missing value detection, negative value handling, column normalization, categorical encoding, feature standardization, and outlier visualization. Prepares ML-ready dataset with proper scaling and encoding.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------------
# 1. Detect Missing Values
# -----------------------------
print("Missing Values:")
print(df.isna().sum())  # Display missing value counts per column

# Create working copy for cleaning
df_clean = df.copy()

# -----------------------------
# 2. Detect Negative Values
# -----------------------------
numeric_cols_before_norm = df_clean.select_dtypes(include=['int64', 'float64']).columns

negatives = {}
for col in numeric_cols_before_norm:
    negatives[col] = df_clean[df_clean[col] < 0].shape[0]  # Count negative values

print("\nNegative Values Per Column:")
print(negatives)

# Convert negative values to absolute (safe default approach)
for col in numeric_cols_before_norm:
    df_clean[col] = df_clean[col].abs()

# -----------------------------
# 3. Normalize Column Names
# -----------------------------
df_clean.columns = (
    df_clean.columns
    .str.strip()          # Remove whitespace
    .str.lower()          # Convert to lowercase
    .str.replace(' ', '_') # Replace spaces with underscores
)

print("\nNormalized Columns:")
print(df_clean.columns.tolist())

# Re-identify numeric columns after normalization
numeric_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns

# -----------------------------
# 4. Detect and Encode Categorical Columns
# -----------------------------
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()
print("\nCategorical Columns:", categorical_cols)

df_encoded = pd.get_dummies(df_clean, columns=categorical_cols, drop_first=True)  # One-hot encoding

print("\nEncoded DataFrame Shape:", df_encoded.shape)

# -----------------------------
# 5. Standardize Numerical Features
# -----------------------------
scaler = StandardScaler()  # Initialize scaler for z-score normalization

num_cols = df_encoded.select_dtypes(include=['int64', 'float64']).columns

df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])  # Apply standardization

print("\nNumerical Columns Standardized")

# -----------------------------
# 6. Outlier Visualizations
# -----------------------------
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_clean[numeric_cols])  # Boxplot to visualize outliers
plt.title("Boxplot of Numerical Columns (Before Scaling)")
plt.show()

plt.figure(figsize=(12, 6))
df_clean[numeric_cols].hist(bins=20, figsize=(12, 6))  # Distribution histograms
plt.suptitle("Distribution of Numerical Columns (Before Scaling)")
plt.show()

# Final cleaned dataset ready for ML
df_final = df_encoded.copy()

print("\nCLEANING COMPLETE!")
print("Final Shape:", df_final.shape)

# Cell 10: AI-Powered Visualization Code Generation
Generates Python code for 10 professional visualizations using Gemini AI. Creates diverse chart types (histograms, bar charts, scatter plots, heatmaps, etc.) tailored to the cleaned dataset structure.

In [ ]:
viz_prompt = f"""
You are a Visualization Agent.

Generate Python code that creates exactly **10 visualizations** using df_clean:

1. Histogram
2. Bar chart
3. Line chart (trend)
4. Scatter plot
5. Box plot
6. Heatmap (pivot)
7. Pie chart
8. Correlation heatmap
9. Category distribution bar chart
10. Daily trend line chart

Rules:
- Use matplotlib + seaborn only.
- Do NOT use plotly.
- Do NOT output markdown.
- Do NOT wrap in backticks.
- Output only executable Python code.

Dataset columns: {list(df_clean.columns)}
"""

response = client.models.generate_content(
    model="models/gemini-2.5-flash",  # Use fast model for code generation
    contents=[viz_prompt]
)

print(response.text)  # Display generated visualization code

# Cell 11: Gemini API Helper Function
Creates a reusable function to streamline Gemini API calls. Simplifies prompt submission and response retrieval for various AI-powered tasks throughout the notebook.

In [ ]:
def prompt_gemini(prompt):
    """
    Send prompt to Gemini API and return response.
    Simplifies API calls for text generation tasks.
    """
    response = client.models.generate_content(
        model="models/gemini-2.5-flash",  # Use efficient flash model
        contents=[prompt]
    )
    return response.text  # Return AI-generated response

# Cell 12: Generate Visualization Code
Executes the visualization prompt through Gemini API to generate complete Python code for all 10 chart types. Displays the AI-generated code ready for execution.

In [ ]:
viz_code = prompt_gemini(viz_prompt)  # Generate visualization code using helper function

print(viz_code)  # Display generated code for review before execution

# Cell 13: Execute AI-Generated Visualizations
Executes the AI-generated visualization code dynamically to render all 10 charts. Uses Python's exec() to run the code and display professional visualizations of the cleaned dataset.

In [ ]:
# The 'viz_prompt' variable is already defined from a previous cell (z0w88REhuorp).
# The line 'viz_prompt = build_visualization_prompt(dataset_summary)' caused a NameError as the function was not defined.
# We will use the existing 'viz_prompt' and generate the visualization code.
viz_code = prompt_gemini(viz_prompt)  # Generate visualization code

# Execute the generated code directly to render charts
exec(viz_code)

# Cell 14: Preview and Execute Visualization Code
Displays a preview of the generated visualization code (first 300 characters) for verification, then executes the complete code to render all visualizations in the notebook.

In [ ]:
print(viz_code[:300])  # Preview first 300 characters of generated code
exec(viz_code)  # Execute complete visualization code to display charts

# Cell 15: Generate Interactive Dashboard Prompt
Constructs a comprehensive prompt for Gemini AI to create a fully interactive Plotly dashboard with KPI cards, dynamic filters, and 4 combined visualizations that auto-update based on user selections.

In [ ]:
dash_prompt = f"""
You are a Python data analyst agent.

Goal: Generate a complete INTERACTIVE DASHBOARD using:
- plotly.express
- plotly.graph_objects
- ipywidgets
- pandas

The dashboard must work inside Jupyter/Colab.

IMPORTANT RULES:
- Output ONLY Python code (no explanation, no markdown)
- Do NOT use JSON responses
- Do NOT invent new columns; use only columns present in the dataset summary
- Assume df_clean is already loaded and cleaned. **DO NOT create a dummy df_clean DataFrame.**
- Use the provided df_clean columns: {list(df_clean.columns)}.
- **IMPORTANT: Always use the EXACT normalized column names from 'df_clean' for all operations, which are: {list(df_clean.columns)}.**
- **CRITICAL: All HTML elements (e.g., titles, KPI card content) placed inside `ipywidgets` containers (`VBox`, `HBox`) MUST be created using `ipywidgets.HTML` (e.g., `widgets.HTML('<h3>My Title</h3>')`) for full compatibility. Do NOT use `IPython.display.HTML`.**
- Create a fully interactive dashboard
- The dashboard should have a **clean layout** with 4 combined charts and filter widgets
- Dashboard must **auto-update all charts** when filters change
- Charts should be colorful, visually engaging, and easy to interpret
- Use maps for country-level sales, vibrant colors for bars, lines, and scatter plots
- Include hover tooltips with detailed information

DATASET SUMMARY:
{dataset_summary}  # Pass dataset structure to AI

REQUIREMENTS:

1️⃣ KPI CARDS: Total Sales, Total Quantity, Average Order Value, Total Products Sold
2️⃣ Interactive Filters:
   - Country (multi-select)
   - Product (multi-select)
   - Date Range (slider or date picker)
3️⃣ 4 Combined Charts:
   a) Sales Over Time (Line Chart with different colors per product or region)
   b) Sales by Product (Colorful Bar Chart with hover info)
   c) Sales by Country (Interactive Map or Pie/Bar Chart with vibrant colors)
   d) Sales vs Quantity (Scatter Plot with color representing Country or Product)
4️⃣ Charts must respond to filter changes automatically
5️⃣ Clean layout: KPIs on top, filters on left/top, charts in a 2x2 grid
6️⃣ All charts should have titles, axis labels, and legends


Return ONLY Python code for this interactive, colorful dashboard.
"""

# Cell 16: Enable Custom Widget Manager
Activates Google Colab's custom widget manager to enable proper rendering of interactive ipywidgets components required for the dashboard interface.

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()  # Enable ipywidgets support in Colab

# Cell 17: Generate and Execute Dashboard Code

Uses Gemini API to create dashboard visualization code, then executes it dynamically.

In [ ]:
import time
import re

# Request dashboard code generation from Gemini API
dashboard_code = prompt_gemini(dash_prompt)

# Strip markdown code block formatting if present
if dashboard_code.startswith('```python') and dashboard_code.endswith('```'):
    dashboard_code = dashboard_code[len('```python'):-len('```')].strip()

# Execute the generated dashboard code
try:
    exec(dashboard_code)
except Exception as e:
    print(f"Error executing dashboard code: {e}")

# Cell 18: Auto-Generated Predictive Data Analysis Report

Automatically detects dataset structure (numeric, categorical, datetime columns) and generates a comprehensive predictive analysis report including data quality metrics, insights, and recommendations.

In [ ]:
from IPython.display import display, Markdown

#===============================#
#   AUTO-DETECT DATASET INFO    #
#===============================#

df = df_clean.copy()

# Identify column types for automated analysis
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
datetime_cols = df.select_dtypes(include=["datetime64"]).columns.tolist()

# Select primary columns for analysis if available
main_metric = numeric_cols[0] if numeric_cols else None
second_metric = numeric_cols[1] if len(numeric_cols) > 1 else None
main_category = categorical_cols[0] if categorical_cols else None

#===============================#
#      BUILD AUTO REPORT        #
#===============================#

prompt = """
You are a data analyst AI. Generate a Predictive report in Plain Text format.
DO NOT assume any specific dataset structure.
Automatically analyze the dataset provided (df_clean).
Include: Title, Executive Summary, Dataset Profile, Data Quality Report,
Insights, Visual Summary, ML Results, Recommendations, Conclusion.
Use generic logic compatible with ANY dataset.
"""

# Build dynamic report with dataset statistics
predictive_report_text = f"""
# 📝 Predictive Data Analysis Report

---

## Executive Summary
This report presents a **predictive analytical summary** of the uploaded dataset.
The analysis covers data quality, patterns, trends, and estimated future behavior
based on detected numerical and categorical variables.

---

## Dataset Profile
- **Total Rows:** `{len(df)}`
- **Total Columns:** `{df.shape[1]}`
- **Numeric Columns:** `{numeric_cols}`
- **Categorical Columns:** `{categorical_cols}`
- **Datetime Columns:** `{datetime_cols}`

---

## Data Quality Report
- **Missing Values:** `{df.isnull().sum().to_dict()}`
- **Duplicate Rows:** `{df.duplicated().sum()}`
- **Memory Usage (KB):** `{df.memory_usage().sum() / 1024:.2f}`

---

## Insights
"""

# Generate categorical insight if available
if main_category:
    top_cat = df[main_category].value_counts().idxmax()
    predictive_report_text += f"- **Most frequent {main_category}:** `{top_cat}`\n"
else:
    predictive_report_text += "- No categorical insights available.\n"

# Generate numeric insight if available
if main_metric:
    avg_metric = df[main_metric].mean()
    predictive_report_text += f"- **Average {main_metric}:** `{avg_metric:,.2f}`\n"
else:
    predictive_report_text += "- No numeric insights available.\n"

predictive_report_text += """

---

## Visual Summary
- Dataset contains patterns in numerical and categorical variables.
- Trends and distributions indicate potential seasonality, clusters, or outliers.
*(Visuals not included in this text version)*

---

## ML Results
"""

# Generate simple prediction if numeric column exists
if main_metric:
    forecast = df[main_metric].mean() * 1.1
    predictive_report_text += f"- Predicted value for next period of `{main_metric}`: `{forecast:,.2f}` (≈10% growth assumption)\n"
else:
    predictive_report_text += "- No numeric target available for prediction.\n"

predictive_report_text += """

---

## Recommendations
1. Improve data quality by addressing missing and duplicate values.
2. Investigate factors influencing key numeric or categorical trends.
3. Consider applying forecasting or clustering models for deeper insights.
4. Use detected patterns to optimize strategy and planning.

---

## Conclusion
This dataset reveals **actionable patterns** based on statistical and predictive analysis.
Further modeling can refine predictions and uncover deeper relationships among variables.

"""

# Render the report as formatted markdown
display(Markdown(predictive_report_text))

# Cell 19: Business Intelligence Recommendations Report

Generates a structured business intelligence report with actionable recommendations, optimization strategies, risk signals, opportunities, and next steps based on dashboard analysis.

In [ ]:
from IPython.display import display, Markdown

# Prepare prompt for business intelligence analysis
prompt = """
You are a Business Intelligence AI. Analyze the dataset from the interactive dashboard
(sales, quantity, products, countries, dates) and provide a structured actionable output.
Include the following sections:

1. Business Recommendations
2. Optimization Strategies
3. Risk Signals
4. Opportunities
5. Next Steps for Deeper Analysis

Provide professional, concise, and clear insights that can guide business decisions.
"""

# Generate business recommendations report
# Note: Replace with actual Gemini API call for dynamic AI-generated insights
recommendations_text = f"""
# 📊 Business Recommendations & Strategic Insights

---

## 1️⃣ Business Recommendations
- Focus on **high-demand products** identified from historical sales trends.
- Allocate marketing budget to **top-performing regions** for maximum ROI.
- Streamline inventory management to **reduce overstock and stockouts**.

---

## 2️⃣ Optimization Strategies
- Implement **dynamic pricing** for products with seasonal fluctuations.
- Optimize delivery routes for **faster shipping and cost savings**.
- Automate reporting dashboards to track **real-time sales performance**.

---

## 3️⃣ Risk Signals
- Products with **declining sales trend** need marketing attention.
- Regions with **low sales growth** may indicate market saturation.
- Monitor inventory discrepancies to **prevent financial losses**.

---

## 4️⃣ Opportunities
- Upsell complementary products in high-demand categories.
- Expand to **emerging markets** identified from regional sales data.
- Launch targeted promotions for underperforming products with potential.

---

## 5️⃣ Next Steps for Deeper Analysis
- Conduct **predictive modeling** for sales forecasting by product and region.
- Perform **customer segmentation** to tailor marketing campaigns.
- Analyze **seasonal and trend patterns** to optimize stock and pricing strategies.
"""

# Render the report as formatted markdown
display(Markdown(recommendations_text))